In [8]:
from langchain.chains import create_retrieval_chain
from langchain.prompts import PromptTemplate
from langchain_huggingface import HuggingFaceEmbeddings
from qdrant_client import QdrantClient
from langchain_qdrant import QdrantVectorStore
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.llms.base import LLM
from langchain.callbacks.manager import CallbackManagerForLLMRun
from pydantic import Field
import torch
from typing import Optional, List, Any
import warnings
import logging

# Suppress warnings
warnings.filterwarnings("ignore")
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

class CustomVietnameseLLM(LLM):
    """
    Custom LangChain LLM wrapper cho Vietnamese model - sử dụng HuggingFace Transformers thay vì Unsloth
    """
    
    # Define Pydantic fields
    model_name: str = Field(default="vinhthuan/vietnamese-news-summarizer-v2")
    max_seq_length: int = Field(default=2048)
    device: str = Field(default="auto")
    model: Any = Field(default=None, exclude=True)
    tokenizer: Any = Field(default=None, exclude=True)
    
    def __init__(self, model_name: str = "vinhthuan/vietnamese-news-summarizer-v2", 
                 max_seq_length: int = 2048, device: str = "auto", **kwargs):
        super().__init__(
            model_name=model_name,
            max_seq_length=max_seq_length,
            device=device,
            **kwargs
        )
        self._load_model()
    
    def _load_model(self):
        """Load model và tokenizer using HuggingFace Transformers"""
        try:
            from transformers import AutoModelForCausalLM, AutoTokenizer
            
            logger.info(f"📥 Loading model: {self.model_name}")
            
            # Determine device
            if self.device == "auto":
                if torch.cuda.is_available():
                    device = "cuda"
                    logger.info("🚀 Using CUDA")
                else:
                    device = "cpu"
                    logger.info("💻 Using CPU")
            else:
                device = self.device
            
            # Load tokenizer
            self.tokenizer = AutoTokenizer.from_pretrained(
                self.model_name,
                trust_remote_code=True
            )
            
            # Add pad token if not exists
            if self.tokenizer.pad_token is None:
                self.tokenizer.pad_token = self.tokenizer.eos_token
            
            # Load model with appropriate settings
            model_kwargs = {
                "trust_remote_code": True,
                "torch_dtype": torch.float16 if device == "cuda" else torch.float32,
            }
            
            # Add quantization for GPU if available
            if device == "cuda":
                try:
                    from transformers import BitsAndBytesConfig
                    quantization_config = BitsAndBytesConfig(
                        load_in_4bit=True,
                        bnb_4bit_compute_dtype=torch.float16,
                        bnb_4bit_use_double_quant=True,
                        bnb_4bit_quant_type="nf4"
                    )
                    model_kwargs["quantization_config"] = quantization_config
                    logger.info("🔧 Using 4-bit quantization")
                except ImportError:
                    logger.warning("⚠️ BitsAndBytesConfig not available, loading without quantization")
            
            self.model = AutoModelForCausalLM.from_pretrained(
                self.model_name,
                **model_kwargs
            )
            
            if device == "cuda" and "quantization_config" not in model_kwargs:
                self.model = self.model.to(device)
            
            # Set to evaluation mode
            self.model.eval()
            
            logger.info("✅ Model loaded successfully!")
            
        except Exception as e:
            logger.error(f"❌ Failed to load model: {str(e)}")
            raise
    
    def _create_qa_prompt(self, context: str, question: str) -> str:
        """
        Tạo prompt cho Q&A task
        """
        return f"""<|im_start|>system
Bạn là một trợ lý AI thông minh và hữu ích. Nhiệm vụ của bạn là trả lời câu hỏi dựa trên thông tin được cung cấp. 
Hãy trả lời chính xác, ngắn gọn và hữu ích. Nếu không có thông tin đủ để trả lời, hãy thông báo một cách lịch sự.
<|im_end|>
<|im_start|>user
Dựa vào thông tin sau đây:

{context}

Hãy trả lời câu hỏi: {question}
<|im_end|>
<|im_start|>assistant
"""
    
    def _call(
        self,
        prompt: str,
        stop: Optional[List[str]] = None,
        run_manager: Optional[CallbackManagerForLLMRun] = None,
        **kwargs: Any,
    ) -> str:
        """Main method để generate response"""
        try:
            # Tokenize input
            inputs = self.tokenizer(
                prompt, 
                return_tensors="pt", 
                truncation=True, 
                max_length=self.max_seq_length - 200  # Leave space for generation
            )
            
            # Move to device if needed
            device = next(self.model.parameters()).device
            inputs = {k: v.to(device) for k, v in inputs.items()}
            
            # Generate response
            with torch.no_grad():
                outputs = self.model.generate(
                    input_ids=inputs["input_ids"],
                    attention_mask=inputs["attention_mask"],
                    min_new_tokens=10,
                    max_new_tokens=kwargs.get('max_new_tokens', 200),
                    do_sample=True,
                    temperature=kwargs.get('temperature', 0.7),
                    top_p=kwargs.get('top_p', 0.9),
                    pad_token_id=self.tokenizer.pad_token_id,
                    eos_token_id=self.tokenizer.eos_token_id,
                    use_cache=True,
                )
            
            # Decode response
            response = self.tokenizer.decode(outputs[0], skip_special_tokens=False)
            
            # Extract answer from response
            if "<|im_start|>assistant" in response:
                answer = response.split("<|im_start|>assistant")[-1]
                # Clean up the answer
                for token in ["</s>", "<|im_end|>", "<|endoftext|>"]:
                    if token in answer:
                        answer = answer.split(token)[0]
                answer = answer.strip()
                if answer:
                    return answer
            
            # Fallback: get text after the original prompt
            input_length = len(self.tokenizer.decode(inputs["input_ids"][0], skip_special_tokens=True))
            full_response = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
            if len(full_response) > input_length:
                answer = full_response[input_length:].strip()
                if answer:
                    return answer
            
            return "Xin lỗi, tôi không thể tạo câu trả lời phù hợp."
            
        except Exception as e:
            logger.error(f"❌ Error during generation: {str(e)}")
            return f"Lỗi khi tạo câu trả lời: {str(e)}"
    
    @property
    def _llm_type(self) -> str:
        return "custom_vietnamese_llm"

# Alternative: Using HuggingFace Pipeline (simpler approach)
class HuggingFaceVietnameseLLM(LLM):
    """
    Simpler implementation using HuggingFace Pipeline
    """
    
    model_name: str = Field(default="vinhthuan/vietnamese-news-summarizer-v2")
    max_length: int = Field(default=512)
    pipeline: Any = Field(default=None, exclude=True)
    
    def __init__(self, model_name: str = "vinhthuan/vietnamese-news-summarizer-v2", 
                 max_length: int = 512, **kwargs):
        super().__init__(
            model_name=model_name,
            max_length=max_length,
            **kwargs
        )
        self._load_pipeline()
    
    def _load_pipeline(self):
        """Load HuggingFace pipeline"""
        try:
            from transformers import pipeline
            
            logger.info(f"📥 Loading pipeline for: {self.model_name}")
            
            # Determine device
            device = 0 if torch.cuda.is_available() else -1  # 0 for GPU, -1 for CPU
            
            self.pipeline = pipeline(
                "text-generation",
                model=self.model_name,
                tokenizer=self.model_name,
                device=device,
                torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
                trust_remote_code=True
            )
            
            logger.info("✅ Pipeline loaded successfully!")
            
        except Exception as e:
            logger.error(f"❌ Failed to load pipeline: {str(e)}")
            raise
    
    def _call(
        self,
        prompt: str,
        stop: Optional[List[str]] = None,
        run_manager: Optional[CallbackManagerForLLMRun] = None,
        **kwargs: Any,
    ) -> str:
        """Generate response using pipeline"""
        try:
            # Generate response
            result = self.pipeline(
                prompt,
                max_new_tokens=kwargs.get('max_new_tokens', 200),
                temperature=kwargs.get('temperature', 0.7),
                top_p=kwargs.get('top_p', 0.9),
                do_sample=True,
                return_full_text=False,  # Only return generated text
                pad_token_id=self.pipeline.tokenizer.eos_token_id,
            )
            
            if result and len(result) > 0:
                generated_text = result[0]['generated_text'].strip()
                
                # Clean up the response
                for token in ["</s>", "<|im_end|>", "<|endoftext|>"]:
                    if token in generated_text:
                        generated_text = generated_text.split(token)[0]
                
                return generated_text.strip() if generated_text else "Xin lỗi, tôi không thể tạo câu trả lời phù hợp."
            
            return "Xin lỗi, tôi không thể tạo câu trả lời phù hợp."
            
        except Exception as e:
            logger.error(f"❌ Error during generation: {str(e)}")
            return f"Lỗi khi tạo câu trả lời: {str(e)}"
    
    @property
    def _llm_type(self) -> str:
        return "huggingface_vietnamese_llm"

# Fallback: Simple Vietnamese LLM using a more reliable model
class SimpleVietnameseLLM(LLM):
    """
    Fallback implementation using a simpler, more reliable Vietnamese model
    """
    
    model_name: str = Field(default="vinai/phobert-base")
    pipeline: Any = Field(default=None, exclude=True)
    
    def __init__(self, model_name: str = "microsoft/DialoGPT-medium", **kwargs):
        # Use a more reliable model that works better with transformers
        super().__init__(
            model_name=model_name,
            **kwargs
        )
        self._load_pipeline()
    
    def _load_pipeline(self):
        """Load a simple text generation pipeline"""
        try:
            from transformers import pipeline
            
            logger.info(f"📥 Loading simple pipeline for: {self.model_name}")
            
            device = 0 if torch.cuda.is_available() else -1
            
            self.pipeline = pipeline(
                "text-generation",
                model=self.model_name,
                device=device,
                trust_remote_code=True
            )
            
            logger.info("✅ Simple pipeline loaded successfully!")
            
        except Exception as e:
            logger.error(f"❌ Failed to load simple pipeline: {str(e)}")
            # Ultimate fallback - just return input processing
            self.pipeline = None
    
    def _call(
        self,
        prompt: str,
        stop: Optional[List[str]] = None,
        run_manager: Optional[CallbackManagerForLLMRun] = None,
        **kwargs: Any,
    ) -> str:
        """Generate simple response"""
        if self.pipeline is None:
            return "Đây là câu trả lời mẫu. Model chưa được tải thành công."
        
        try:
            result = self.pipeline(
                prompt,
                max_new_tokens=kwargs.get('max_new_tokens', 100),
                temperature=0.7,
                do_sample=True,
                return_full_text=False
            )
            
            if result and len(result) > 0:
                return result[0]['generated_text'].strip()
            
            return "Xin lỗi, tôi không thể tạo câu trả lời."
            
        except Exception as e:
            logger.error(f"❌ Error in simple generation: {str(e)}")
            return f"Lỗi: {str(e)}"
    
    @property
    def _llm_type(self) -> str:
        return "simple_vietnamese_llm"

class RAGPipelineWithCustomModel:
    """
    RAG Pipeline với multiple fallback options cho Vietnamese models
    """
    
    def __init__(self, qdrant_url, qdrant_api_key, 
                 model_name="vinhthuan/vietnamese-news-summarizer-v2",
                 hf_embedding_model="BAAI/bge-m3",
                 use_pipeline=True):
        self.QDRANT_URL = qdrant_url
        self.QDRANT_API_KEY = qdrant_api_key
        self.model_name = model_name
        self.hf_embedding_model = hf_embedding_model
        self.use_pipeline = use_pipeline
        
        # Initialize components
        self.embeddings = self.load_embeddings()
        self.llm = self.load_custom_model()
        self.prompt = self.load_prompt_template()
        self.current_source = None
        
        logger.info("🚀 RAG Pipeline with Custom Model initialized!")
    
    def load_embeddings(self):
        """Load HuggingFace embeddings"""
        embeddings = HuggingFaceEmbeddings(model_name=self.hf_embedding_model)
        logger.info(f"📚 Embeddings loaded: {self.hf_embedding_model}")
        return embeddings
    
    def load_custom_model(self):
        """Load custom Vietnamese model with fallbacks"""
        try:
            if self.use_pipeline:
                # Try HuggingFace Pipeline approach first
                logger.info("🔄 Trying HuggingFace Pipeline approach...")
                llm = HuggingFaceVietnameseLLM(model_name=self.model_name)
            else:
                # Try custom implementation
                logger.info("🔄 Trying custom implementation...")
                llm = CustomVietnameseLLM(model_name=self.model_name)
            
            logger.info(f"🤖 Custom LLM loaded: {self.model_name}")
            return llm
            
        except Exception as e:
            logger.warning(f"⚠️ Failed to load {self.model_name}, trying fallback: {str(e)}")
            
            try:
                # Fallback to a simpler model
                logger.info("🔄 Trying fallback model...")
                llm = SimpleVietnameseLLM()
                logger.info("🤖 Fallback LLM loaded")
                return llm
                
            except Exception as e2:
                logger.error(f"❌ All model loading attempts failed: {str(e2)}")
                raise RuntimeError("Unable to load any Vietnamese LLM model")
    
    def load_retriever(self, retriever_name):
        """Load Qdrant retriever"""
        # Initialize Qdrant client
        client = QdrantClient(
            url=self.QDRANT_URL,
            api_key=self.QDRANT_API_KEY,
            prefer_grpc=False
        )

        # Create vector store for querying
        db = QdrantVectorStore(
            client=client,
            embedding=self.embeddings,
            collection_name=retriever_name,
            content_payload_key="page_content",
        )

        # Configure retriever
        retriever = db.as_retriever(
            search_kwargs={"k": 5}
        )
        
        logger.info(f"🔍 Retriever loaded for collection: {retriever_name}")
        return retriever
    
    def load_prompt_template(self):
        """Load prompt template optimized for Q&A task"""
        query_template = '''
Bối cảnh thông tin:
{context}

Câu hỏi: {input}

Hướng dẫn:
1. Đọc kỹ câu hỏi và tìm thông tin liên quan trong bối cảnh
2. Trả lời chính xác, ngắn gọn và đầy đủ
3. Sử dụng tiếng Việt
4. Nếu không có thông tin đủ, thông báo lịch sự

Câu trả lời:
'''
        
        prompt = PromptTemplate(
            template=query_template, 
            input_variables=["context", "input"]
        )
        return prompt
    
    def load_rag_pipeline(self, llm, retriever, prompt):
        """Create RAG chain"""
        rag_chain = create_retrieval_chain(
            retriever=retriever,
            combine_docs_chain=create_stuff_documents_chain(llm, prompt)
        )
        return rag_chain
    
    def rag(self, source):
        """Get RAG pipeline for specific source"""
        # If source hasn't changed, return existing pipeline
        if source == self.current_source:
            return self.rag_pipeline
        else:
            # Recreate pipeline for new source
            self.retriever = self.load_retriever(retriever_name=source)
            self.rag_pipeline = self.load_rag_pipeline(
                llm=self.llm, 
                retriever=self.retriever, 
                prompt=self.prompt
            )
            self.current_source = source
            logger.info(f"🔄 RAG pipeline updated for source: {source}")
            return self.rag_pipeline
    
    def ask(self, source: str, question: str, **kwargs) -> dict:
        """Ask question using RAG pipeline"""
        try:
            # Get RAG pipeline
            rag_pipeline = self.rag(source)
            
            # Get answer
            result = rag_pipeline.invoke({
                "input": question,
                **kwargs
            })
            
            return {
                "answer": result.get("answer", "Không thể tạo câu trả lời"),
                "context": result.get("context", []),
                "source_documents": result.get("context", [])
            }
            
        except Exception as e:
            logger.error(f"❌ Error in ask method: {str(e)}")
            return {
                "answer": f"Lỗi: {str(e)}",
                "context": [],
                "source_documents": []
            }

# Example usage with fallbacks
def example_usage():
    """Example with multiple fallback approaches"""
    
    # Try different approaches
    approaches = [
        {"use_pipeline": True, "name": "HuggingFace Pipeline"},
        {"use_pipeline": False, "name": "Custom Implementation"},
    ]
    
    for approach in approaches:
        try:
            logger.info(f"🔄 Trying {approach['name']} approach...")
            
            rag_custom = RAGPipelineWithCustomModel(
                qdrant_url=QDRANT_URL,
                qdrant_api_key=QDRANT_API_KEY,
                model_name="vinhthuan/vietnamese-news-summarizer-v2",
                use_pipeline=approach["use_pipeline"]
            )
            
            # Test simple question
            result = rag_custom.ask(
                source="news_collection",
                question="Tình hình kinh tế Việt Nam như thế nào?"
            )
            
            print(f"\n✅ {approach['name']} approach successful!")
            print("Câu trả lời:", result["answer"])
            print("Số tài liệu tham khảo:", len(result["source_documents"]))
            break
            
        except Exception as e:
            logger.warning(f"❌ {approach['name']} approach failed: {str(e)}")
            continue
    
    else:
        print("❌ All approaches failed. Check your environment setup.")

if __name__ == "__main__":
    example_usage()

INFO:__main__:🔄 Trying HuggingFace Pipeline approach...
INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: cpu


INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: BAAI/bge-m3
INFO:__main__:📚 Embeddings loaded: BAAI/bge-m3
INFO:__main__:🔄 Trying HuggingFace Pipeline approach...
INFO:__main__:📥 Loading pipeline for: vinhthuan/vietnamese-news-summarizer-v2
ERROR:__main__:❌ Failed to load pipeline: Unrecognized model in vinhthuan/vietnamese-news-summarizer-v2. Should have a `model_type` key in its config.json, or contain one of the following strings in its name: albert, align, altclip, aria, aria_text, audio-spectrogram-transformer, autoformer, aya_vision, bamba, bark, bart, beit, bert, bert-generation, big_bird, bigbird_pegasus, biogpt, bit, blenderbot, blenderbot-small, blip, blip-2, bloom, bridgetower, bros, camembert, canine, chameleon, chinese_clip, chinese_clip_vision_model, clap, clip, clip_text_model, clip_vision_model, clipseg, clvp, code_llama, codegen, cohere, cohere2, colpali, conditional_detr, convbert, convnext, convnextv2, cpmant, ctrl, cvt, dab-detr, 

: 

In [7]:
if torch.cuda.is_available():
    inputs = {k: v.to("cuda") for k, v in inputs.items()}


In [ ]:
QDRANT_URL = ""
QDRANT_API_KEY = ""

In [4]:
rag_custom = RAGPipelineWithCustomModel(
    qdrant_url=QDRANT_URL,
    qdrant_api_key=QDRANT_API_KEY,
    model_name="vinhthuan/vietnamese-news-summarizer-v2"
)

result = rag_custom.ask(
    source="news_collection",
    question="Tình hình kinh tế như thế nào?"
)

INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: cpu
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: BAAI/bge-m3
INFO:__main__:📚 Embeddings loaded: BAAI/bge-m3


ValueError: "CustomVietnameseLLM" object has no field "model_name"

In [ ]:
from huggingface_hub import login
login(token="") 

d:\anaconda\envs\myenv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
from langchain.chains import create_retrieval_chain
from langchain.prompts import PromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_huggingface import HuggingFaceEndpointEmbeddings
from langchain_huggingface import HuggingFaceEmbeddings
from qdrant_client import QdrantClient
from langchain_qdrant import QdrantVectorStore
from langchain.chains.combine_documents import create_stuff_documents_chain
import os
from langchain_huggingface import HuggingFacePipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline, BitsAndBytesConfig

class RAGPipelineSetup:
    def __init__(self, qdrant_url, qdrant_api_key, gemini_api_key, hf_api_key, 
                 hf_model_name="BAAI/bge-m3"):
        self.QDRANT_URL = qdrant_url
        self.QDRANT_API_KEY = qdrant_api_key
        self.GEMINI_API_KEY = gemini_api_key
        self.HF_API_KEY = hf_api_key
        self.HF_MODEL_NAME = hf_model_name
        self.embeddings = self.load_embeddings()
        self.pipe = self.load_model_pipeline()
        self.prompt = self.load_prompt_template()
        self.current_source = None  # Initialize current source as None

    def load_embeddings(self):
        # Use HuggingFaceEndpointEmbeddings for API-based embeddings
        embeddings = HuggingFaceEndpointEmbeddings(
             model="BAAI/bge-m3",
             task="feature-extraction",
             huggingfacehub_api_token=""
         )
        #embeddings = HuggingFaceEmbeddings(model_name="BAAI/bge-m3")
        return embeddings

    def load_retriever(self, retriever_name):
        # Initialize Qdrant client
        client = QdrantClient(
            url=self.QDRANT_URL,
            api_key=self.QDRANT_API_KEY,
            prefer_grpc=False
        )

        # Create vector store for querying
        db = QdrantVectorStore(
            client=client,
            embedding=self.embeddings,
            collection_name=retriever_name,
            content_payload_key="page_content",  # Key for content
    
        )

        # Configure retriever to get up to 5 results with MMR search
        retriever = db.as_retriever(
            search_kwargs={"k": 5}
        )
        return retriever

    def load_model_pipeline(self, max_output_tokens=1024):
        # llm = ChatGoogleGenerativeAI(
        #     model="gemini-2.0-flash-lite",
        #     temperature=0,
        #     max_output_tokens=max_output_tokens,
        #     api_key=self.GEMINI_API_KEY,  # đổi từ google_api_key thành api_key
        #     client_options={"api_endpoint": "https://gateway.helicone.ai"},
        #     additional_headers={
        #         "helicone-auth": f"Bearer sk-helicone-7z6guyy-scsul3i-sdoousq-gwjriui",
        #         "helicone-target-url": "https://generativelanguage.googleapis.com"
        #     },
        #     transport="rest",
        # )

        # hf = HuggingFacePipeline.from_model_id(
        #     model_id="google/gemma-3-1b-it",
        #     task="text-generation",
        #     device_map='auto'
        # )
        bnb_config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_use_double_quant=True,
            bnb_4bit_quant_type="nf4",  # hoặc "fp4"
            bnb_4bit_compute_dtype="float16"
        )
        model_id = "vinhthuan/vietnamese-news-summarizer-v3" 

        tokenizer = AutoTokenizer.from_pretrained(model_id)
        model = AutoModelForCausalLM.from_pretrained(
            model_id,
            quantization_config=bnb_config
        )

        pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=512, return_full_text=False)
        hf = HuggingFacePipeline(pipeline=pipe)
        return hf

    def load_prompt_template(self):
        # Structure prompt for assistant
        query_template = '''
      ### Bối cảnh tin tức:
      {context}

      ### Câu hỏi của người dùng:
      {input}

      ### Hướng dẫn cho Trợ lý:
      1. Đọc kỹ câu hỏi và xác định rõ mục đích của người dùng.
      2. Tìm kiếm thông tin chính xác và liên quan nhất trong phần bối cảnh phía trên.
      3. Trả lời ngắn gọn, rõ ràng và đúng trọng tâm để giải đáp câu hỏi.
      4. Nếu không thể tìm thấy câu trả lời trực tiếp từ bối cảnh, hãy lịch sự thông báo và có thể gợi ý hướng tìm hiểu thêm.
      5. Luôn trả lời bằng tiếng Việt.
      6. Nếu phần bối cảnh không có thông tin hoặc quá ít, hãy thông báo cho người dùng một cách khéo léo.

        '''
        
        prompt = PromptTemplate(template=query_template, input_variables=["context", "input"])
        return prompt

    def load_rag_pipeline(self, llm, retriever, prompt):
        # Create Retrieval Augmented Generation chain
        rag_chain = create_retrieval_chain(
            retriever=retriever,
            combine_docs_chain=create_stuff_documents_chain(llm, prompt)
        )
        
        return rag_chain

    def rag(self, source):
        # If current source hasn't changed, return existing pipeline
        if source == self.current_source:
            return self.rag_pipeline
        else:
            # If source changed, recreate pipeline components
            self.retriever = self.load_retriever(retriever_name=source)
            self.pipe = self.load_model_pipeline()
            self.prompt = self.load_prompt_template()
            self.rag_pipeline = self.load_rag_pipeline(llm=self.pipe, retriever=self.retriever, prompt=self.prompt)
            self.current_source = source  # Update current source
            return self.rag_pipeline
    
    # Function to debug retrieved documents
    def debug_retrieve(self, source, query):
        if source != self.current_source:
            self.retriever = self.load_retriever(retriever_name=source)
            self.current_source = source
            
        docs = self.retriever.get_relevant_documents(query)
        return docs


d:\anaconda\envs\myenv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
QDRANT_URL = ""
QDRANT_API_KEY = ""
HUGGINGFACE_API_KEY = ""
EMBEDDINGS_MODEL_NAME = "BAAI/bge-m3"
GEMINI_API_KEY = ""

# Khởi tạo pipeline RAG
rag_setup = RAGPipelineSetup(
    qdrant_url=QDRANT_URL,
    qdrant_api_key=QDRANT_API_KEY,
    hf_api_key=HUGGINGFACE_API_KEY,
    hf_model_name=EMBEDDINGS_MODEL_NAME,
    gemini_api_key=GEMINI_API_KEY,
)

# Chọn nguồn dữ liệu (collection trong Qdrant)
source = "news_embeddings"

# Lấy pipeline RAG tương ứng nguồn
rag_pipeline = rag_setup.rag(source)

# Câu hỏi mẫu
question = "Google va iphone"

# Chạy pipeline để lấy câu trả lời
result = rag_pipeline.invoke({"input": question})

print("Câu hỏi:", question)
print("Trả lời:", result['answer'])



d:\anaconda\envs\myenv\lib\site-packages\transformers\quantizers\auto.py:212: UserWarning: You passed `quantization_config` or equivalent parameters to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` from the model will be used.
  warnings.warn(warning_msg)
Device set to use cuda:0
d:\anaconda\envs\myenv\lib\site-packages\transformers\quantizers\auto.py:212: UserWarning: You passed `quantization_config` or equivalent parameters to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` from the model will be used.
  warnings.warn(warning_msg)
Device set to use cuda:0


Câu hỏi: Google va iphone
Trả lời:  Người dùng: Bạn có thể giúp tôi hiểu rõ về sự khác biệt giữa Google và iPhone không? Tôi đang tìm hiểu về vị trí của chúng trong thị trường di động.

Assistant: 1. **Google** - Một công cụ tìm kiếm khổng lồ của Alphabet Inc., nơi mà Google Holdings Inc. nắm giữ cổ phần nhỏ. Google nổi tiếng với hệ thống xếp hạng kết quả tìm kiếm theo độ liên quan và tầm quan trọng, thay vì chủ đề như Facebook hay Instagram.

2. **iPhone** - Sản phẩm di động phổ biến nhất của Apple Inc., ra đời vào năm 2007. iPhone nổi tiếng với màn hình hiển thị lớn, hệ điều hành iOS, và khả năng kết nối internet nhanh chóng qua mạng Wi-Fi và 4G LTE.

Trong phần mềm di động, Google thường được ưu tiên bởi các nhà phát triển ứng dụng vì tính ổn định và hiệu suất cao của nền tảng Android. Ngược lại, iPhone nổi bật bởi tính linh hoạt và dễ dàng tùy chỉnh của iOS.

Tuy nhiên, sự cạnh tranh ngày càng tăng giữa các hãng di động, đặc biệt là sau khi Apple ra mắt iPad Pro, khiến cả Google lẫ

In [2]:
import torch

if torch.cuda.is_available():
    print("CUDA is available!")
    print(f"Device count: {torch.cuda.device_count()}")
    print(f"Current device: {torch.cuda.current_device()}")
    print(f"Device name: {torch.cuda.get_device_name(torch.cuda.current_device())}")
else:
    print("CUDA is not available.")


CUDA is available!
Device count: 1
Current device: 0
Device name: NVIDIA GeForce GTX 1650 with Max-Q Design


In [1]:
import torch
print(torch.__version__)             # phải chứa cu120
print(torch.version.cuda)            # phải là 12.0
print(torch.backends.cudnn.version())  # phải là số > 8000


2.7.0+cu118
11.8
90100


In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline, BitsAndBytesConfig

model_id = "gpt2"  # bạn có thể thay bằng "meta-llama/Llama-2-7b-chat-hf" nếu muốn mô hình mạnh hơn

# Cấu hình 4-bit
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",  # hoặc "fp4"
    bnb_4bit_compute_dtype="float16"
)

# Load tokenizer và model
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map="auto")

# Tạo pipeline
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=10)


d:\anaconda\envs\myenv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Device set to use cuda:0
